# imports

In [1]:
import os, json
import openai
from dotenv import dotenv_values, load_dotenv
from openai.types.chat.chat_completion import ChatCompletion
from IPython.display import display, Markdown


# load api key

In [2]:
load_dotenv()
client:openai.OpenAI = openai.OpenAI()

# functions
## openai helper function

In [3]:
def get_llm_response(client:openai.OpenAI, prompt:str, model:str='gpt-4o-mini', return_dict_body:bool=False, return_response_body:bool=False) -> str | dict | ChatCompletion:
    response = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        model=model,
    )
    
    # print(type(response))
    
    if return_response_body:
        return response
    if return_dict_body:
        return response.model_dump()
    return response.choices[0].message.content


## prompt template interpolate function

In [4]:
def get_colors_prompt(prompt:str) -> str:
    return f'''
    You are a color palette generating assistant that responds to text prompts for color palettes
    Your should generate color palettes that fit the theme, mood, or instructions in the prompt.
    The palettes should be between 2 and 8 colors, the more the better.

    Q: Convert the following verbal description of a color palette into a list of colors: The Mediterranean Sea
    A: ["#006699", "#66CCCC", "#F0E68C", "#008000", "#F08080"]

    Q: Convert the following verbal description of a color palette into a list of colors: sage, nature, earth
    A: ["#EDF1D6", "#9DC08B", "#609966", "#40513B"]


    Desired Format: just a JSON array of hexadecimal color codes, nothing else before or after

    Q: Convert the following verbal description of a color palette into a list of colors: {prompt}
    A:    
    '''


## notebook markdown display helper

In [5]:
def display_colors(colors, width:int=4):
    display(
        Markdown(
            ' '.join(
                f'<span style="color: {color}">{chr(9608) * width}</span>'
                for color in colors
            )
        )
    )

# main
## get json color array

In [6]:
response = get_llm_response(
    client, 
    get_colors_prompt('blue ocean')
)
response

'["#003F5C", "#2A7D8E", "#4FB3BF", "#A8D8EA", "#F1FAEE"]'

## convert json to python array

In [7]:
colors = json.loads(response)
colors

['#003F5C', '#2A7D8E', '#4FB3BF', '#A8D8EA', '#F1FAEE']

## display result

In [8]:
display_colors(colors)

<span style="color: #003F5C">████</span> <span style="color: #2A7D8E">████</span> <span style="color: #4FB3BF">████</span> <span style="color: #A8D8EA">████</span> <span style="color: #F1FAEE">████</span>

# macro helper functions

In [9]:
def get_color_palette(client:openai.OpenAI, prompt:str) -> list:
    userprompt = get_colors_prompt(prompt)
    response = get_llm_response(client, userprompt)
    colors = json.loads(response)
    return colors


In [10]:
from json import JSONDecodeError


def show_palette(client:openai.OpenAI, prompt:str):
    try:
        colors = get_color_palette(client, prompt)
        display_colors(colors)
        print(colors)
    except JSONDecodeError as jde:
        print(f'something wrong with llm response format: {response!r}')

In [11]:
show_palette(client, 'ocean abyss')

<span style="color: #001F3F">████</span> <span style="color: #004D6D">████</span> <span style="color: #013B48">████</span> <span style="color: #005B77">████</span> <span style="color: #0096A6">████</span> <span style="color: #3CC3D3">████</span>

['#001F3F', '#004D6D', '#013B48', '#005B77', '#0096A6', '#3CC3D3']


In [12]:
show_palette(client, 'cyber terminal with green chars')

<span style="color: #00FF00">████</span> <span style="color: #000000">████</span> <span style="color: #1C1C1C">████</span> <span style="color: #33CC33">████</span> <span style="color: #66FF66">████</span> <span style="color: #009900">████</span>

['#00FF00', '#000000', '#1C1C1C', '#33CC33', '#66FF66', '#009900']


In [15]:
show_palette(client, 'spongebob')

<span style="color: #F9D94B">████</span> <span style="color: #CD4E3D">████</span> <span style="color: #3B7E9A">████</span> <span style="color: #A7C6ED">████</span> <span style="color: #FFC107">████</span> <span style="color: #FF6F20">████</span>

['#F9D94B', '#CD4E3D', '#3B7E9A', '#A7C6ED', '#FFC107', '#FF6F20']


In [16]:
show_palette(client, 'superman')

<span style="color: #C8102E">████</span> <span style="color: #F0F0F0">████</span> <span style="color: #004B87">████</span>

['#C8102E', '#F0F0F0', '#004B87']


In [17]:
show_palette(client, 'european tabby cat white gray and black')

<span style="color: #FFFFFF">████</span> <span style="color: #B0B0B0">████</span> <span style="color: #404040">████</span> <span style="color: #000000">████</span> <span style="color: #AFAFAF">████</span>

['#FFFFFF', '#B0B0B0', '#404040', '#000000', '#AFAFAF']
